<h2>Using a Neural Network to fit the California Housing data</h2>

# Data Preparation

In [98]:
# California Housing dataset
import urllib.request
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# load data from csv file
urllib.request.urlretrieve("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv", "housing.csv")
housing = pd.read_csv('housing.csv')

# Using the setting inplace=False, drop() creates a copy of the data and does not affect housing dataset
housing_data = housing.drop("median_house_value", axis=1, inplace=False)
housing_target = housing["median_house_value"].copy()
feature_names = list(housing_data.columns)

#  Transformation pipeline at https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, feature_names[:-1]),
    ('cat', OneHotEncoder(), [feature_names[-1]]),
])

housing_preprocessed = full_pipeline.fit_transform(housing_data)
print(housing_preprocessed.shape)

(20640, 13)


In [99]:
X = housing_preprocessed
y = housing_target.to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# (for comparsion) Using scikit-learn's Linear Regression model to fit the data

In [100]:
from sklearn.linear_model import LinearRegression

# documentation at https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
lr = LinearRegression()

lr.fit(X_train, y_train)
print("model training error : %.3f" % lr.score(X_train, y_train))
print("model testing error: %.3f" % lr.score(X_test, y_test))

model training error : 0.640
model testing error: 0.666


# Using PyTorch nn.Sequential() to build a neural network to fit the data

In [101]:
import torch
from torch import nn
from torch import optim

#converts x_train, x_test, y_train, y_test into tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).squeeze() 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).squeeze()

#creating Sequential model
model = nn.Sequential(
    nn.Linear(13, 64),
    nn.Tanh(),
    nn.Linear(64,1),
)

#Loss function
loss_fn = nn.MSELoss()
#set max number of iterations
n_epochs = 5000
#optimizer
optimizer = optim.SGD(model.parameters(), lr=0.0001)

#training loop
for epoch in range(1, n_epochs + 1):
    # Set training mode
    model.train() 
    y_pred_train = model(X_train_tensor)
    loss_train = loss_fn(y_pred_train.squeeze(), y_train_tensor)

    # Backward pass
    loss_train.backward()
    # Update weights
    optimizer.step()
    # Zero gradients
    optimizer.zero_grad()

    #prints the training and validation loss
    if epoch == 1 or epoch % 100 == 0:
        # Set evaluation mode
        model.eval()
        y_pred_val = model(X_test_tensor)
        loss_val = loss_fn(y_pred_val.squeeze(), y_test_tensor)  

        print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, Validation loss {loss_val.item():.4f}")

# Set evaluation mode
model.eval()

#calculates test loss  
y_pred_test = model(X_test_tensor)
loss_test = loss_fn(y_pred_test.squeeze(), y_test_tensor) 

print(f"Test loss: {loss_test.item():.4f}")


Epoch 1, Training loss 56069459968.0000, Validation loss 56157859840.0000
Epoch 100, Training loss 16399368192.0000, Validation loss 16369868800.0000
Epoch 200, Training loss 10471561216.0000, Validation loss 10440920064.0000
Epoch 300, Training loss 8339233280.0000, Validation loss 8283198464.0000
Epoch 400, Training loss 7417796096.0000, Validation loss 7304110592.0000
Epoch 500, Training loss 6658002432.0000, Validation loss 6543735808.0000
Epoch 600, Training loss 6082086912.0000, Validation loss 6003782144.0000
Epoch 700, Training loss 5741580288.0000, Validation loss 5602222592.0000
Epoch 800, Training loss 5474632704.0000, Validation loss 5359897600.0000
Epoch 900, Training loss 5291606016.0000, Validation loss 5117635072.0000
Epoch 1000, Training loss 5137945600.0000, Validation loss 4947048448.0000
Epoch 1100, Training loss 4956376064.0000, Validation loss 4799183360.0000
Epoch 1200, Training loss 4848791552.0000, Validation loss 4620240896.0000
Epoch 1300, Training loss 47437

# Subclassing nn.Module to build a neural network to fit the data

In [102]:
import torch
from torch import nn
from torch import optim

class SubclassNetwork(nn.Module):
    def __init__(self, set_size):
        super().__init__()  # <1>
        
        self.hidden_linear = nn.Linear(set_size, 64)
        self.hidden_activation = nn.Tanh()
        self.output_linear = nn.Linear(64, 1)
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = self.hidden_activation(hidden_t)
        output_t = self.output_linear(activated_t)
        
        return output_t
    
set_size = X_train.shape[1]  
model = SubclassNetwork(set_size)

#training loop function
def trainingLoop(model, X_train, X_test, y_train, y_test, opti, n_epochs):
    for epoch in range(1, n_epochs + 1):
    # Set training mode
        model.train() 
        y_pred_train = model(X_train)
        loss_train = loss_fn(y_pred_train.squeeze(), y_train)  # Squeeze the predictions
    # Backward pass
        loss_train.backward()
    # Update weights
        opti.step()
    # Zero gradients
        opti.zero_grad()

        #prints training and validation loss
        if epoch == 1 or epoch % 100 == 0:
        # Set evaluation mode
            model.eval()
            y_pred_val = model(X_test)
            loss_val = loss_fn(y_pred_val.squeeze(), y_test)  # Squeeze the predictions
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f}, Validation loss {loss_val.item():.4f}")

#loss function
loss_fn = nn.MSELoss()
#optimizer
optimizer = optim.SGD(model.parameters(), lr=0.0001)

#converts x_train, x_test, y_train, y_test into tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).squeeze() 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).squeeze()
trainingLoop(model, X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, optimizer, 5000)

#set model to evaluation mode
model.eval()

#calculates test loss  
y_pred_test = model(X_test_tensor)
loss_test = loss_fn(y_pred_test.squeeze(), y_test_tensor)  # Squeeze the predictions

print(f"Test loss: {loss_test.item():.4f}")


Epoch 1, Training loss 56069574656.0000, Validation loss 56181940224.0000
Epoch 100, Training loss 16129658880.0000, Validation loss 16072100864.0000
Epoch 200, Training loss 9838605312.0000, Validation loss 9807795200.0000
Epoch 300, Training loss 7613194240.0000, Validation loss 7558524928.0000
Epoch 400, Training loss 6527055360.0000, Validation loss 6406428672.0000
Epoch 500, Training loss 5899644928.0000, Validation loss 5764470272.0000
Epoch 600, Training loss 5520250368.0000, Validation loss 5344396800.0000
Epoch 700, Training loss 5207925760.0000, Validation loss 5069363200.0000
Epoch 800, Training loss 4962328064.0000, Validation loss 4839010816.0000
Epoch 900, Training loss 4825259008.0000, Validation loss 4614091264.0000
Epoch 1000, Training loss 4650229248.0000, Validation loss 4448231936.0000
Epoch 1100, Training loss 4515095040.0000, Validation loss 4375314432.0000
Epoch 1200, Training loss 4408342528.0000, Validation loss 4182234112.0000
Epoch 1300, Training loss 4328048